In [1]:
import matplotlib.pyplot as plt
import vtk
import numpy as np
import sys
import math
import os
import glob
from vtk.util.numpy_support import *
import pandas
from multiprocessing import Pool

In [2]:
def read_vti(filename):
    reader = vtk.vtkXMLImageDataReader()
    reader.SetFileName(filename)
    reader.Update()
    return reader.GetOutput()

def write_vti(filename,data):
    writer = vtk.vtkXMLImageDataWriter()
    writer.SetInputData(data)
    writer.SetFileName(filename)
    writer.Update()
    
def write_vtu(filename,data):
    writer = vtk.vtkXMLUnstructuredGridWriter()
    writer.SetInputData(data)
    writer.SetFileName(filename)
    writer.Update()    
    
def write_multiblock(filename,data):
    writer = vtk.vtkXMLMultiBlockDataWriter()
    writer.SetInputData(data)
    writer.SetFileName(filename)
    writer.Update()    

In [3]:
input_path = '../out/mfix_local_grid/slic_compare_100.vti'
confidence_th = 0.9
size_threshold = 100

In [5]:
data = read_vti(input_path)
data.GetPointData().SetActiveScalars('feature_similarity')

1

In [9]:
thresholding = vtk.vtkThreshold()
thresholding.ThresholdByUpper( confidence_th )
thresholding.SetInputData(data)


seg = vtk.vtkConnectivityFilter()
seg.SetInputConnection(thresholding.GetOutputPort())
seg.SetExtractionModeToLargestRegion()
seg.Update()

print (seg.GetRegionIdAssignmentMode())

write_vtu('/Users/sdutta/Desktop/test.vtu', seg.GetOutput())

segmentation = vtk.vtkConnectivityFilter()
segmentation.SetInputConnection(thresholding.GetOutputPort())
segmentation.SetExtractionModeToAllRegions()
segmentation.ColorRegionsOn()
segmentation.Update()

ug = segmentation.GetOutput()
num_segments = segmentation.GetNumberOfExtractedRegions()
print ("Extracted regions have " + str(num_segments) + " segments..")

voxel_counts = np.zeros(num_segments)
for i in range(ug.GetPointData().GetArray('RegionId').GetNumberOfTuples()):
    regionId = int(ug.GetPointData().GetArray('RegionId').GetTuple(i)[0])
    voxel_counts[regionId] = voxel_counts[regionId]+1

## largest region
empty_space_id = np.argmax(voxel_counts)
print ('region to be excluded ' + str(empty_space_id))

## exclude the largest region
thresholding1 = vtk.vtkThreshold()
thresholding1.ThresholdByUpper( confidence_th )
thresholding1.SetInputData(data)

mb = vtk.vtkMultiBlockDataSet()
mb.SetNumberOfBlocks(num_segments-1)

idx=0
for i in range(num_segments):
    
    if i != int(empty_space_id) and voxel_counts[i] > size_threshold:
        print (i, idx)
        thresholding2 = vtk.vtkThreshold()
        thresholding2.SetInputData(ug)
        thresholding2.ThresholdBetween(i,i)
        thresholding2.Update()
        
        mb.SetBlock(idx,thresholding2.GetOutput())
        idx=idx+1
        
write_multiblock('/Users/sdutta/Desktop/test.vtm', mb)

0
Extracted regions have 28 segments..
region to be excluded 1
0 0
2 1
3 2
12 3
15 4
20 5
21 6
22 7
23 8
24 9
25 10


In [11]:
## combine a few vtp files into a single file
def read_vtp(filename):
    reader = vtk.vtkXMLPolyDataReader()
    reader.SetFileName(filename)
    reader.Update()
    return reader.GetOutput()

def write_vtp(filename,data):
    writer = vtk.vtkXMLPolyDataWriter()
    writer.SetInputData(data)
    writer.SetFileName(filename)
    writer.Update()
    
pdata = vtk.vtkPolyData()
pts = vtk.vtkPoints()

for filename in glob.glob('/Users/sdutta/Desktop/insitu_data_55000_out/*.vtp'):
    data = read_vtp(filename)
    for i in range(data.GetNumberOfPoints()):
        pts.InsertNextPoint(data.GetPoint(i))
        
pdata.SetPoints(pts)        
        
write_vtp('/Users/sdutta/Desktop/combined.vtp',pdata)       